# Foreground Galaxy Identification for FRB Sample (v2.0)

This notebook identifies foreground galaxies for a sample of 12 FRBs by querying astronomical catalogs and calculating physical impact parameters. It utilizes the modular `v2.0` galaxy search toolkit.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord

# Add the project root to sys.path to import the v2_0 module
module_path = os.path.abspath(os.path.join('..', '..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Import v2_0 components
from galaxies.v2_0.config import TARGETS, DEFAULT_IMPACT_KPC
from galaxies.v2_0.utils import parse_coord, get_angular_radius, calculate_impact_parameter
from galaxies.v2_0.engines import NedEngine, VizierEngine

print(f"Loaded {len(TARGETS)} targets.")

## 2. Loading FRB Sample Coordinates

We load the metadata for the 12 FRBs into a pandas DataFrame for easier manipulation.

In [ ]:
frb_df = pd.DataFrame(TARGETS, columns=['ra_str', 'dec_str', 'z_frb'])
frb_df['target_id'] = range(1, len(frb_df) + 1)

# Parse coordinates to decimal degrees
coords = [parse_coord(r, d) for r, d in zip(frb_df.ra_str, frb_df.dec_str)]
frb_df['ra_deg'] = [c.ra.deg for c in coords]
frb_df['dec_deg'] = [c.dec.deg for c in coords]

frb_df.head()

## 3. Querying Galaxy Catalogs

We use the `NedEngine` and `VizierEngine` to search for galaxies around each FRB. For this demonstration, we'll search around the first target.

In [ ]:
# Select first target
target = frb_df.iloc[0]
coord = parse_coord(target.ra_str, target.dec_str)
radius = get_angular_radius(target.z_frb, DEFAULT_IMPACT_KPC)

print(f"Searching around Target 1 (z={target.z_frb}) with radius {radius:.2f}")

# Initialize engines
ned = NedEngine()
# Query NED
ned_results = ned.query(coord, radius)

print(f"Found {len(ned_results)} results in NED.")
ned_results.head()

## 4. Calculating Angular and Physical Separations

We calculate the physical impact parameter $b$ for each galaxy using the angular diameter distance $d_A$ at the galaxy's redshift.

In [ ]:
if not ned_results.empty:
    ned_results['impact_kpc'] = ned_results.apply(
        lambda row: calculate_impact_parameter(
            row['ra'], row['dec'], row['z'], target.ra_deg, target.dec_deg
        ), axis=1
    )
    
ned_results.head()

## 5. Identifying Foreground Galaxy Candidates

We filter for galaxies where $z_{gal} < z_{frb}$ and the impact parameter is within our threshold (e.g., 100 kpc).

In [ ]:
foreground_galaxies = ned_results[
    (ned_results['z'] < target.z_frb) & 
    (ned_results['impact_kpc'] <= DEFAULT_IMPACT_KPC)
]

print(f"Found {len(foreground_galaxies)} foreground galaxy candidates.")
foreground_galaxies.head()

## 6. Visualizing FRB-Galaxy Environments

We plot the distribution of galaxies around the FRB sightline.

In [ ]:
plt.figure(figsize=(8, 6))
plt.scatter(ned_results.ra, ned_results.dec, c=ned_results.z, cmap='viridis', label='All Galaxies')
plt.scatter(target.ra_deg, target.dec_deg, marker='*', s=200, color='red', label='FRB Sightline')
plt.colorbar(label='Redshift')
plt.xlabel('RA (deg)')
plt.ylabel('Dec (deg)')
plt.title(f'Galaxy Environment around Target 1')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 7. Exporting Foreground Galaxy Catalog

Finally, we save the identified foreground galaxies to a CSV file.

In [ ]:
if not foreground_galaxies.empty:
    foreground_galaxies.to_csv('foreground_galaxies_v2.csv', index=False)
    print("Saved foreground galaxies to foreground_galaxies_v2.csv")